In [1]:
import numpy as np

In [2]:
a = [1, 2, 3]
b = [[20, 25, 27],[29, 33, 40]]

In [3]:
np.exp(a)

array([ 2.71828183,  7.3890561 , 20.08553692])

In [4]:
np.exp(b)

array([[4.85165195e+08, 7.20048993e+10, 5.32048241e+11],
       [3.93133430e+12, 2.14643580e+14, 2.35385267e+17]])

In [5]:
np.sum(np.exp(b))

2.356044462894082e+17

In [6]:
np.sum(np.exp(a))

30.19287485057736

In [7]:
a/np.sum(np.exp(a))

array([0.0331204 , 0.06624079, 0.09936119])

In [8]:
2*np.array(b)

array([[40, 50, 54],
       [58, 66, 80]])

In [9]:
import random
import numpy as np
from ift725.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [10]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'ift725/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
    # subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]

    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image

    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


In [11]:
from ift725.classifiers.softmax import softmax_naive_loss_function
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001

In [12]:
loss = 0.0
dW = np.zeros_like(W)

In [13]:
X = X_dev 
loss = loss*0
dW = dW*0
for i in range(X.shape[0]):
    score = np.dot(X[i], W)
    scores = X[i].dot(W)


In [14]:
scores = X.dot(W)    #(500, 3073) W:(3073, 10)

In [15]:
scores

array([[-0.06789915,  0.03818789, -0.06511912, ..., -0.04686903,
        -0.03390238, -0.22482216],
       [ 0.42214871,  0.20394371,  0.52156072, ..., -0.8029443 ,
        -0.38287595, -0.35250865],
       [ 0.46443409, -0.32407803, -0.0042067 , ...,  0.26715697,
         0.2867795 , -0.47205649],
       ...,
       [-0.14087122,  0.18566817, -0.03947593, ...,  0.25055737,
         0.28615404,  0.16164036],
       [ 0.79140096, -0.06721619, -0.22042473, ...,  0.29501253,
         0.2485909 , -0.08184917],
       [-0.45585104, -0.01388331, -0.05151171, ...,  0.61088984,
        -0.32474096,  0.09636352]])

In [16]:
y = y_dev
y

array([1, 7, 5, 9, 8, 9, 2, 8, 0, 2, 3, 4, 1, 0, 4, 1, 5, 9, 7, 4, 1, 5,
       6, 6, 6, 9, 1, 4, 8, 1, 8, 3, 1, 3, 5, 6, 6, 6, 4, 9, 3, 7, 9, 4,
       7, 9, 3, 5, 4, 0, 3, 0, 2, 9, 3, 7, 3, 2, 9, 5, 1, 2, 2, 3, 7, 4,
       6, 3, 1, 1, 7, 3, 0, 3, 7, 9, 1, 2, 7, 4, 4, 6, 5, 7, 3, 6, 6, 5,
       4, 5, 2, 2, 1, 6, 4, 2, 2, 9, 6, 5, 5, 5, 7, 3, 5, 2, 5, 4, 2, 3,
       8, 5, 3, 4, 6, 1, 1, 6, 3, 2, 5, 7, 9, 6, 6, 5, 0, 4, 3, 9, 4, 7,
       1, 0, 6, 7, 6, 3, 0, 3, 1, 3, 6, 7, 0, 6, 5, 3, 7, 4, 8, 1, 5, 3,
       4, 1, 9, 7, 3, 8, 9, 6, 1, 4, 3, 4, 7, 3, 1, 3, 5, 7, 6, 2, 7, 6,
       3, 0, 1, 2, 2, 6, 7, 6, 6, 4, 5, 8, 1, 8, 7, 4, 7, 6, 7, 3, 8, 6,
       3, 9, 9, 3, 2, 2, 9, 7, 9, 8, 5, 4, 7, 0, 0, 9, 7, 4, 4, 9, 4, 4,
       9, 1, 5, 5, 5, 6, 9, 8, 4, 7, 4, 8, 0, 4, 7, 0, 3, 8, 1, 1, 1, 1,
       1, 5, 7, 6, 9, 0, 9, 6, 5, 1, 1, 0, 6, 5, 2, 8, 0, 2, 5, 8, 8, 5,
       0, 7, 8, 4, 8, 4, 2, 1, 9, 8, 8, 9, 8, 4, 9, 7, 6, 0, 8, 6, 9, 2,
       0, 5, 6, 5, 4, 9, 2, 8, 7, 2, 6, 3, 6, 2, 0,

In [17]:
correct = scores[range(X.shape[0]), y]
correct.shape

(500,)

In [18]:
correct[:5]

array([ 0.03818789, -0.8029443 , -0.2122885 , -0.22129068,  0.39125593])

In [19]:
correct = correct.reshape(X.shape[0], 1)
correct.shape

(500, 1)

In [20]:
margin = scores - correct - 1
margin

array([[-1.10608704, -1.        , -1.10330701, ..., -1.08505692,
        -1.07209027, -1.26301005],
       [ 0.22509301,  0.00688801,  0.32450501, ..., -1.        ,
        -0.57993165, -0.54956436],
       [-0.32327741, -1.11178953, -0.7919182 , ..., -0.52055452,
        -0.500932  , -1.25976798],
       ...,
       [-1.51514098, -1.1886016 , -1.4137457 , ..., -1.12371239,
        -1.08811572, -1.2126294 ],
       [ 0.44819646, -0.41042069, -0.56362923, ..., -0.04819196,
        -0.09461359, -0.42505367],
       [-1.48330182, -1.04133408, -1.07896249, ..., -0.41656094,
        -1.35219174, -0.93108725]])

In [21]:
dW = np.zeros(W.shape)
margin[margin>0] = 1

In [23]:
margin_count = np.sum(margin, axis=1)
margin[range(X.shape[0]),y] -= margin_count

In [25]:
margin.shape

(500, 10)

In [26]:
margin

array([[-1.10608704,  9.70757968, -1.10330701, ..., -1.08505692,
        -1.07209027, -1.26301005],
       [ 1.        ,  1.        ,  1.        , ..., -0.49363048,
        -0.57993165, -0.54956436],
       [-0.32327741, -1.11178953, -0.7919182 , ..., -0.52055452,
        -0.500932  , -1.25976798],
       ...,
       [-1.51514098, -1.1886016 , -1.4137457 , ..., -1.12371239,
        -1.08811572, -1.2126294 ],
       [ 1.        , -0.41042069, -0.56362923, ..., -0.04819196,
        -0.09461359, -0.42505367],
       [-1.48330182, -1.04133408, -1.07896249, ..., -0.41656094,
        -1.35219174, -0.93108725]])

In [28]:
dW = X.T.dot(margin)

In [30]:
dW/X.shape[0]

array([[ 5.34743460e-01,  2.87943989e+00, -1.41735939e+01, ...,
         1.06981454e+01,  1.16131906e+01,  1.17870880e+01],
       [ 1.21843931e+01, -1.70214020e+00, -1.11924326e+01, ...,
         1.43880817e+01,  2.44062649e+01,  1.68614163e+01],
       [ 2.69095245e+01, -5.02383784e+00, -1.61449562e+01, ...,
         2.13788916e+01,  4.01773702e+01,  2.96501844e+01],
       ...,
       [-1.40139783e+01, -1.49345313e+01, -1.97497964e-01, ...,
         2.80404426e+01, -2.47861520e+01,  2.08844591e+01],
       [-8.18335189e+00, -6.25317861e+00, -1.02692985e+01, ...,
         1.57933472e+01, -5.78456473e+00,  2.57639283e+01],
       [-2.90776033e-01,  1.31102499e-01, -2.42596006e-03, ...,
        -1.32235808e-01, -4.75115865e-02, -1.60350528e-01]])

In [31]:
reg = 2e5
dW += reg*2*W

In [32]:
dW

array([[ 2.31984731e+02,  1.43526716e+03, -7.04251668e+03, ...,
         5.35991837e+03,  5.85786137e+03,  5.94419573e+03],
       [ 6.11783137e+03, -8.81501092e+02, -5.65456630e+03, ...,
         7.26892024e+03,  1.22435432e+04,  8.43093056e+03],
       [ 1.34090564e+04, -2.54470251e+03, -8.07685369e+03, ...,
         1.06637777e+04,  2.00451987e+04,  1.47830186e+04],
       ...,
       [-6.99001360e+03, -7.48974264e+03, -4.32226925e+01, ...,
         1.40319800e+04, -1.24053400e+04,  1.04523734e+04],
       [-4.08006693e+03, -3.14575940e+03, -5.06468075e+03, ...,
         7.86541837e+03, -2.85223920e+03,  1.28318082e+04],
       [-1.63507295e+02,  5.24776269e+01,  3.73265410e+01, ...,
         2.79738797e+00, -2.09069451e+01, -7.23187283e+00]])